### tester différentes méthodes de detection de pic sur toute les masses, pour pouvoir tester DBSCAN et comparer les méthodes

In [ ]:
from skimage.feature import peak_local_max
from sklearn.cluster import DBSCAN
import skimage
import baseline_correction
from read_chroma import read_chromato_and_chromato_cube
import numpy as np
from sklearn.preprocessing import StandardScaler
import pandas as pd
import projection
import plot

In [ ]:
file_path = '/home/camille/Documents/app/data/input/A-F-028-817822-droite-ReCIVA.cdf'
# estimation with filtering_factor_estimation.ipynb
mod_time = 1.7
eps = 0.2

In [ ]:
chromato_tic_preprocessed, time_rn, chromato_cube_preprocessed, sigma, mass_range = read_chromato_and_chromato_cube(file_path, mod_time, pre_process=True)

In [ ]:
# filtering_factor = 50 #TODO
# sigma = skimage.restoration.estimate_sigma(chromato_tic_preprocessed, channel_axis=None)
# dynamic_threshold_fact = filtering_factor * sigma * 100 / np.max(chromato_tic_preprocessed)
# dynamic_threshold_fact

# peak_detection.peak_local_max : mode = mass per mass

In [ ]:
coordinates_all_mass = []

## plm_mas_per_mass_multiprocess

for mass in range(len(chromato_cube_preprocessed)):
# for mass in range(100,150):
    
    #without multiprocess, peak local max
    coordPLM = skimage.feature.peak_local_max(chromato_cube_preprocessed[mass], min_distance=1, threshold_rel=0.2)
    # coordPERS= pers_hom_kernel(i,m_chromato=chromato_cube_preprocessed[i], min_persistence=0.02,threshold_abs=0.2)
    # for x,y in coord:
    for x, y in coordPLM:
        coordinates_all_mass.append([mass, x, y])
        # df = pd.DataFrame(coordinates_all_mass, columns=["mass","x", "y"])
        # df.to_csv("coordinates_peaks.csv", index=False)
        
    
## save output
coordinates_all_mass = np.delete(coordinates_all_mass, 0, -1) # supp l'info de la mass
len(coordinates_all_mass)




In [ ]:
#clustering
#TODO scaled value before DBSCAN
coordinates_all_mass_scaled = StandardScaler().fit_transform(coordinates_all_mass)

clustering = DBSCAN(eps=eps, min_samples=5).fit(coordinates_all_mass_scaled)


In [ ]:
np.max(clustering.labels_)

In [ ]:
# nombre de clusters détectés (sans le label -1)
#TODO a corriger ds le clustering , enelever les points de bruit + construire la liste des clusters sans les bruits
n_clusters = len(set(clustering.labels_)) - (1 if -1 in clustering.labels_ else 0)
# initialisation de la liste des clusters
clusters = [[] for _ in range(n_clusters)]


for i, (t1, t2) in enumerate(coordinates_all_mass):
    label = clustering.labels_[i]
    if label != -1: # enleve les points de bruit
        clusters[label].append([t1, t2])

print("nb de clusters", len(clusters))
n_noise = np.sum(clustering.labels_ == -1)
print(f"Nombre de points considérés comme du bruit : {n_noise}")


In [ ]:
# Coordinates of the geometrical center point in the cluster for every clusters
coordinates = []
for cluster in clusters:
    if not cluster:
        print("Erreur, cluster vide")
    elif (len(cluster) > 1):
        # point le + intense:
        # coord = cluster[np.argmax(np.array([chromato[coord[0], coord[1]] for coord in cluster]))] 

        arr = np.array(cluster)
        # centre geometrique du cluster
        median_x = np.median(arr[:, 0]) 
        median_y = np.median(arr[:, 1])
        coord=[round(median_x),round(median_y)]
    else:
        coord = cluster[0]

    coordinates.append(coord)
peaks_coordinates = np.array(coordinates)

peaks_coordinates

In [ ]:
coordinates_in_chromato = projection.matrix_to_chromato(
    peaks_coordinates, 
    time_rn, 
    mod_time, 
    chromato_tic_preprocessed.shape
    )
plot.visualizer((chromato_tic_preprocessed, time_rn), mod_time, title="chromato", log_chromato=False, points=coordinates_in_chromato)  